# 03 - .Net Interactive Notebook 

Dit is een .Net Interactive Notebook in FSharp. Kwa idee is dit sterk gebaseerd op *Jupyter Notebook*, maar helemaal gericht op .Net programmeren.

## Hoe werkt een .Net Interactive Notebook
Je kunt code-cells en markdown-cells maken. De code cells kun je uitvoeren door met de muis links van 
de code-cell te hoveren, en te klikken op een driehoekig play-button te klikken. Als je met de muis 
hovert tussen de horizontale ruimte tussen twee cells, dan kun je een nieuw code- of markdown blokje toevoegen.

Je kunt ook bovenaan op ``Run All`` klikken, maar voor de workshop is een laag tempo beter, en kun je het beter cell-voor-cell uitvoeren.
Als je opnieuw wilt beginnen, kun je ``Clear Outputs`` klikken.

## In deze workshop
We doorlopen hier een korte demonstratie van hoe FSharp gebruikt kan worden voor Data-Analyse.

Hierbij laten we sterk zien, de rol van **Type Providers**. Dit zijn libraries die on-the-fly types kunnen genereren, die een bepaalde input
modelleren. Bij een Database wordt het schema omgezet in types, en kan zo de data uitgelezen worden. 
Met CSV en JSON type providers (niet in deze workshop) worden de types bepaald op basis van sample data en [type inference](https://en.wikipedia.org/wiki/Type_inference). 

Het extra wat deze Notebook biedt is de integratie van code, markdown en grafische visualisatie. Type Providers zijn ook beschikbaar in de Fsharp interactive en gecompileerde code.

In [ ]:
#r "nuget: FSharp.Data, 4.2.3"
#r "nuget: Plotly.NET, 2.0.0-preview.9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.9"
#r "nuget: SQLProvider, 1.2.9"

open FSharp.Data.Sql
open Plotly.NET
open Plotly.NET.Interactive
open FSharp.Data

## World Bank Type Provider

Een FSharp Type Provider is een adapter op data, uit verschillende bronnen, waarvan on-the-fly 
de data-types worden gecreeerd. 

De World Bank Provider is een voorgebakken Type Provider voor Demo's en workshops, de hele api-interface zit er al in.

We beginnen hiermee dan ook mee wegens de eenvoudige demonstratie. 

In [ ]:
let wb = WorldBankData.GetDataContext()

wb
  .Countries.Netherlands
  .Indicators.``Gross capital formation (% of GDP)``
|> Seq.maxBy fst


In het volgende stukje code worden verschillende data-bronnen parallel opgehaald met Async. Je kunt Async ook voor andere toepassingen gebruiken.

In [ ]:
let countries =
 [| 
    wb.Countries.Brazil
    wb.Countries.Canada
    wb.Countries.France
    wb.Countries.Greece
    wb.Countries.``Low income``
    wb.Countries.``High income``
    wb.Countries.``United Kingdom``
    wb.Countries.``United States`` |]

let gcf =
    [ for c in countries ->
        async { return c, c.Indicators.``Gross capital formation (% of GDP)`` } ]
    |> Async.Parallel
    |> Async.RunSynchronously
    |> List.ofArray
    |> List.map(fun (c, ind) -> c.Name, ind.Values |> List.ofSeq |> List.sum)

Chart.Column (gcf |> List.map snd, gcf |> List.map fst)
|> Chart.withTitle "World Bank"
|> Chart.withXAxisStyle ("Staat", ShowGrid=false)
|> Chart.withYAxisStyle ("Gross capital formation (% of GDP)", ShowGrid=false)

## SQL Server Type Provider

Hieronder wordt een demonstratie gegeven van een SQL Server Type Provider.

Bij het verbinden naar een database, wordt het schema naar data-types vertaald,
en deze zijn direct beschikbaar, zonder compilatie stap.

Ter voorbereiding, doe het volgende:
- Zorg ervoor dat je de laatste versie hebt van Docker Desktop
- Open Docker Desktop (dashboard) op tabblad "Containers/Apps"
- Open de CLI van de Sql Server Container (via het icoontje)
- Voer het volgende commande uit in het CLI-window: ./entrypoint

Bovenstaande stappen initialiseert database "Northwind". Nota bene - deze initialisatie is nodig iedere keer als je de container opnieuw opstart, dus ook als je VS Code opnieuw opstart.

Hieronder wordt een verbinding gemaakt met SQL Server, die in een andere container draait.

In [ ]:
let [<Literal>] connectionString = "Server=sqlserver,1433;Database=Northwind;User Id=sa;Password=SQLServer2017"
type sql = SqlDataProvider< 
              ConnectionString = connectionString,
              DatabaseVendor = Common.DatabaseProviderTypes.MSSQLSERVER,
              IndividualsAmount = 1000,
              UseOptionTypes = true >
              

In [ ]:
let ctx = sql.GetDataContext()

Vanaf dit punt kun je bij de database objecten via:
ctx.Dbd. + druk op Ctrl+Enter

Probeer dit eens in een nieuw code-blok.

In [ ]:
//  data ophalen uit de DB en printen
ctx.Dbo.Categories
|>  List.ofSeq
|>  List.take 5
|>  List.iter(fun i -> 
    match i.Description with
    |   Some d -> printfn "%s" d
    |   None -> printfn "<leeg>"
    )

Onderstaande leest de Customer tabel, groepeert het aantal per (bekende) Staat, en maakt hiervan een grafiek.

In [ ]:
let data = 
    ctx.Dbo.Customers
    |>  List.ofSeq
    |>  List.groupBy(fun rc -> rc.Region)
    |>  List.map(fun (go, rs) -> go, rs.Length)
    |>  List.map(fun (go, rs) -> go |> Option.map(fun g -> g, rs))
    |>  List.choose id

let keys = data |> List.map fst
let values = data |> List.map snd

Chart.Column (values, keys)
|> Chart.withTitle "Aantal customers per Staat"
|> Chart.withXAxisStyle ("Staat", ShowGrid=false)
|> Chart.withYAxisStyle ("Aantal Customers", ShowGrid=false)